# DL-GENAi PROJECT — Scratch BiLSTM
# Name  : Abhishek Saha
# Roll  : 23f1001572
# Model : Scratch BiLSTM 

## IMPORTS AND SETUP

In [19]:
import os
import random
import math
import time
import json
import re
import html
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import wandb
!wandb login 20d9b18a55f275c39d05bf53e51e8b328aeffff5

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

## CONFIGURATION

In [20]:
class CFG:
    # dataset paths 
    TRAIN_PATH = "/kaggle/input/2025-sep-dl-gen-ai-project/train.csv"
    TEST_PATH  = "/kaggle/input/2025-sep-dl-gen-ai-project/test.csv"
    SAMPLE_SUB = "/kaggle/input/2025-sep-dl-gen-ai-project/sample_submission.csv"

    # model / train hyperparams
    seed = 42
    device = "cuda" if torch.cuda.is_available() else "cpu"
    max_len = 100
    min_freq = 2
    embedding_dim = 128
    hidden_dim = 384            
    num_layers = 2              
    dropout = 0.3
    num_heads = 4
    batch_size = 64
    lr = 3e-4
    epochs = 15
    weight_decay = 1e-6
    clip_grad = 1.0
    scheduler = "cosine"        
    warmup_steps = 100
    save_path = "scratch_bilstm_final.pth"
    project = "23f1001572-t32025"
    run_name = "scratch-bilstm-v2"

CFG = CFG()
TARGET_COLS = ["anger", "fear", "joy", "sadness", "surprise"]

In [21]:
def set_seed(s=CFG.seed):
    random.seed(s)
    np.random.seed(s)
    torch.manual_seed(s)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(s)

set_seed()

## Data Loading

In [22]:
train = pd.read_csv(CFG.TRAIN_PATH)
test  = pd.read_csv(CFG.TEST_PATH)

TARGET_COLS = ["anger","fear","joy","sadness","surprise"]
print("Train shape:", train.shape, "Test shape:", test.shape)

Train shape: (6827, 8) Test shape: (1707, 2)


## Preprocessing

In [23]:
contraction_map = {
    "n't":" not", "'re":" are", "'s":" is", "'d":" would", "'ll":" will", "'ve":" have", "'m":" am"
}

def basic_text_preprocessing(text):
    if pd.isna(text):
        return ""
    s = html.unescape(str(text))
    s = s.lower()

    s = re.sub(r"http\S+|www\.\S+", " ", s)
    s = re.sub(r"@\w+", " ", s)
    s = re.sub(r"#(\w+)", r"\1", s)
    s = re.sub(r"[^a-z0-9!?'\s]", " ", s)

    
    s = re.sub(r"(.)\1{2,}", r"\1\1", s)

    
    s = re.sub(r"\s+", " ", s).strip()

    return s



train["clean_text"] = train["text"].apply(basic_text_preprocessing)
test["clean_text"]  = test["text"].apply(basic_text_preprocessing)


## VOCAB + ENCODING

In [24]:
def build_vocab(texts, min_freq=CFG.min_freq):
    freq = defaultdict(int)
    for t in texts:
        for w in t.split():
            freq[w] += 1
    vocab = {"<pad>":0, "<unk>":1}
    for w,c in sorted(freq.items(), key=lambda x: -x[1]):
        if c >= min_freq:
            vocab[w] = len(vocab)
    return vocab

vocab = build_vocab(train["clean_text"].values, min_freq=CFG.min_freq)
vocab_size = len(vocab)
print("Vocab size:", vocab_size)

def encode(text, max_len=CFG.max_len):
    tokens = text.split()
    ids = [vocab.get(w, vocab["<unk>"]) for w in tokens[:max_len]]
    if len(ids) < max_len:
        ids += [vocab["<pad>"]] * (max_len - len(ids))
    return ids


Vocab size: 5510


## Data loader

In [25]:
train_df, val_df = train_test_split(train, test_size=0.1, random_state=CFG.seed, shuffle=True, stratify=None)

train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)

train_df["input_ids"] = train_df["clean_text"].apply(lambda x: encode(x, CFG.max_len))
val_df["input_ids"]   = val_df["clean_text"].apply(lambda x: encode(x, CFG.max_len))
test["input_ids"]     = test["clean_text"].apply(lambda x: encode(x, CFG.max_len))

class EmotionDataset(Dataset):
    def __init__(self, df, targets=True):
        self.ids = df["input_ids"].tolist()
        self.targets = df[TARGET_COLS].values.astype("float32") if targets else None

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        x = torch.tensor(self.ids[idx], dtype=torch.long)
        if self.targets is None:
            return x
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        return x, y

train_dataset = EmotionDataset(train_df, targets=True)
val_dataset   = EmotionDataset(val_df, targets=True)
test_dataset  = EmotionDataset(test, targets=False)

train_loader = DataLoader(train_dataset, batch_size=CFG.batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2, pin_memory=True)


## MODEL ARCHITECTURE — Scratch BiLSTM

In [26]:
class ScratchBiLSTMAttn(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels, 
                 num_layers=1, dropout=0.4, num_heads=4 ):
        super().__init__()

        # Embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        
        self.spatial_dropout = nn.Dropout2d(dropout)

        # BiLSTM
        self.lstm = nn.LSTM(
            embed_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True
        )

        
        self.gru = nn.GRU(
            hidden_dim * 2,
            hidden_dim,
            num_layers=1,
            batch_first=True,
            bidirectional=True
        )

        # Multi-Head Attention
        self.mha = nn.MultiheadAttention(
            embed_dim=hidden_dim * 2,
            num_heads=num_heads,
            batch_first=True
        )

        self.norm1 = nn.LayerNorm(hidden_dim * 2)
        self.norm2 = nn.LayerNorm(hidden_dim * 2)

        self.dropout = nn.Dropout(dropout)

        
        self.fc = nn.Linear(hidden_dim * 4, num_labels)

    def forward(self, input_ids):
        x = self.embedding(input_ids)

        
        x = self.spatial_dropout(x.unsqueeze(0)).squeeze(0)

        # LSTM
        out, _ = self.lstm(x)
        out = self.norm1(out)

        # GRU (stacked)
        out, _ = self.gru(out)
        out = self.norm2(out)

        # Attention
        att_out, _ = self.mha(out, out, out)
        out = out + att_out

        # Pooling
        avg_pool = torch.mean(out, 1)
        max_pool, _ = torch.max(out, 1)

        out = torch.cat((avg_pool, max_pool), dim=1)

        out = self.dropout(out)
        logits = self.fc(out)
        return logits


In [ ]:
model = ScratchBiLSTMAttn(
    vocab_size=len(vocab),
    embed_dim=CFG.embedding_dim,
    hidden_dim=CFG.hidden_dim,
    num_labels=len(TARGET_COLS),
    num_layers=CFG.num_layers,
    num_heads=CFG.num_heads,
    dropout=CFG.dropout
).to(CFG.device)

## LOSS FUNCTION + OPTIMIZER

In [28]:
label_counts = train[TARGET_COLS].sum().values + 1e-6
pos_weight = torch.tensor((len(train) - label_counts) / (label_counts + 1e-6), dtype=torch.float32).to(CFG.device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)

In [29]:

def get_cosine_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps, num_cycles=0.5, last_epoch=-1):
    
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / max(1.0, float(num_warmup_steps))
        progress = float(current_step - num_warmup_steps) / max(1, float(num_training_steps - num_warmup_steps))
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)

total_steps = int(len(train_loader) * CFG.epochs)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=CFG.warmup_steps, num_training_steps=total_steps)


## Wandb log

In [12]:
wandb.init(project=CFG.project, name=CFG.run_name, config={
    "model": "ScratchBiLSTMAttn",
    "vocab_size": vocab_size,
    "embedding_dim": CFG.embedding_dim,
    "hidden_dim": CFG.hidden_dim,
    "num_layers": CFG.num_layers,
    "num_heads": CFG.num_heads,
    "batch_size": CFG.batch_size,
    "lr": CFG.lr,
    "epochs": CFG.epochs,
    "max_len": CFG.max_len
})
wandb.watch(model, log="all", log_freq=100)

wandb: Currently logged in as: abhisheksaha (abhisheksahaiitm) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## Training and validation

In [30]:
def train_one_epoch(model, loader, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    for batch in tqdm(loader, desc="Train"):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        logits = model(x)
        loss = criterion(logits, y)
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), CFG.clip_grad)
        optimizer.step()
        scheduler.step()
        running_loss += loss.item()
    return running_loss / len(loader)

def evaluate(model, loader, device):
    model.eval()
    total_loss = 0.0
    preds_list = []
    labels_list = []
    with torch.no_grad():
        for batch in tqdm(loader, desc="Val"):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            total_loss += loss.item()
            probs = torch.sigmoid(logits).cpu().numpy()
            preds_list.append(probs)
            labels_list.append(y.cpu().numpy())
    preds = np.vstack(preds_list)
    labels = np.vstack(labels_list)

    # find best threshold per class by searching
    best_thresholds = []
    best_f1s = {}
    for i, col in enumerate(TARGET_COLS):
        best_f1 = 0.0
        best_t = 0.5
        for t in np.linspace(0.1, 0.9, 17):
            pbin = (preds[:, i] > t).astype(int)
            f1 = f1_score(labels[:, i], pbin, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_t = t
        best_thresholds.append(best_t)
        best_f1s[col] = best_f1

    macro_f1 = np.mean(list(best_f1s.values()))
    return total_loss / len(loader), macro_f1, best_f1s, best_thresholds

## Training loop

In [14]:
best_f1 = -1.0
best_thresholds = [0.5] * len(TARGET_COLS)

for epoch in range(CFG.epochs):
    start = time.time()
    train_loss = train_one_epoch(model, train_loader, optimizer, scheduler, CFG.device)
    val_loss, val_macro_f1, per_f1s, thresholds = evaluate(model, val_loader, CFG.device)

    elapsed = time.time() - start
    print(f"Epoch {epoch+1}/{CFG.epochs} - train_loss: {train_loss:.4f} val_loss: {val_loss:.4f} val_macro_f1: {val_macro_f1:.4f} time: {elapsed:.1f}s")
    print("Per-class F1:", per_f1s)
    print("Thresholds:", thresholds)

    # W&B logging
    log_dict = {
        "epoch": epoch+1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "val_macro_f1": val_macro_f1,
        "lr": scheduler.get_last_lr()[0]
    }
    for k,v in per_f1s.items():
        log_dict[f"f1_{k}"] = v
    for i,t in enumerate(thresholds):
        log_dict[f"thr_{TARGET_COLS[i]}"] = float(t)

    wandb.log(log_dict)

    # save best
    if val_macro_f1 > best_f1:
        best_f1 = val_macro_f1
        best_thresholds = thresholds
        torch.save({
            "model_state_dict": model.state_dict(),
            "vocab": vocab,
            "best_thresholds": best_thresholds,
            "config": CFG.__dict__,
            "best_f1": best_f1
        }, CFG.save_path)
        print(f"Saved new best model with F1: {best_f1:.4f}")

# final log
wandb.log({"best_val_f1": best_f1})
wandb.finish()

Train:   0%|          | 0/96 [00:00<?, ?it/s]

Val:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 1/12 - train_loss: 1.2672 val_loss: 1.3606 val_macro_f1: 0.4676 time: 10.3s
Per-class F1: {'anger': 0.2542372881355932, 'fear': 0.7293023255813954, 'joy': 0.38534278959810875, 'sadness': 0.47032474804031354, 'surprise': 0.4985590778097983}
Thresholds: [0.85, 0.1, 0.1, 0.1, 0.9]
Saved new best model with F1: 0.4676


Train:   0%|          | 0/96 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a6ef4d80680>
Exception ignored in: Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__
<function _MultiProcessingDataLoaderIter.__del__ at 0x7a6ef4d80680>
    Traceback (most recent call last):
self._shutdown_workers()  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1618, in __del__

      File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
self._shutdown_workers()
      File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
if w.is_alive():
     if w.is_alive():   
     ^ ^^ ^^ ^ ^^ ^^^^^^^
^  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    ^^assert self._parent_pid == os.getpid(), 'can only test a child process'
^^  ^^ ^ ^ 
   File "/usr/lib/

...

Epoch 11/12 - train_loss: 0.0833 val_loss: 1.9133 val_macro_f1: 0.7413 time: 10.7s
Per-class F1: {'anger': 0.7000000000000001, 'fear': 0.8133848133848135, 'joy': 0.7261146496815286, 'sadness': 0.7383177570093459, 'surprise': 0.7285067873303168}
Thresholds: [0.85, 0.45000000000000007, 0.85, 0.65, 0.25]
Saved new best model with F1: 0.7413


Train:   0%|          | 0/96 [00:00<?, ?it/s]

Val:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 12/12 - train_loss: 0.0752 val_loss: 1.9292 val_macro_f1: 0.7418 time: 10.8s
Per-class F1: {'anger': 0.7051282051282051, 'fear': 0.8114143920595533, 'joy': 0.7261146496815286, 'sadness': 0.7370892018779343, 'surprise': 0.7293064876957495}
Thresholds: [0.25, 0.35, 0.8, 0.6, 0.2]
Saved new best model with F1: 0.7418


best_val_f1,▁
epoch,▁▂▂▃▄▄▅▅▆▇▇█
f1_anger,▁▃▄▇▇▇▇▇████
f1_fear,▁▁▃▅▄▅▆▆▇▇██
f1_joy,▁▂▃▆▇█▇█████
f1_sadness,▁▁▄▅▅▇▇▇████
f1_surprise,▁▅▅▆▇▇██████
lr,███▇▆▅▄▃▂▂▁▁
thr_anger,▇▅▇▇▇███▅▃▇▁
thr_fear,▁▅█▄▅▃▄█▅▅▇▅
thr_joy,▁▄▄▇▆▇█▇█▇█▇


## INFERENCE ON TEST

In [17]:
print("Loading best model for inference...")

import numpy as np
import torch.serialization

torch.serialization.add_safe_globals([np.core.multiarray.scalar])


ckpt = torch.load(
    CFG.save_path,
    map_location=CFG.device,
    weights_only=False
)

model.load_state_dict(ckpt["model_state_dict"])
model.to(CFG.device)
model.eval()


best_thresholds = np.array(ckpt.get("best_thresholds", [0.5]*len(TARGET_COLS)))


all_preds = []
with torch.no_grad():
    for x in tqdm(test_loader, desc="Test"):
        x = x.to(CFG.device)
        logits = model(x)
        probs = torch.sigmoid(logits).cpu().numpy()
        all_preds.extend(probs)

all_preds = np.array(all_preds)
binary_preds = (all_preds > best_thresholds).astype(int)


Loading best model for inference...


Test:   0%|          | 0/27 [00:00<?, ?it/s]

## Submission

In [18]:
submission = pd.DataFrame({
    "id": test["id"],
    "anger": binary_preds[:,0],
    "fear": binary_preds[:,1],
    "joy": binary_preds[:,2],
    "sadness": binary_preds[:,3],
    "surprise": binary_preds[:,4],
})
submission.to_csv("submission.csv", index=False)
print("Saved submission_scratch_bilstm.csv")


Saved submission_scratch_bilstm.csv
